In [1]:
import os
import pickle

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from src.data_preparation.data_preprocessing import DataReshaperLSTM
from src.visualization.charts import ModelCharts
from src.visualization.metrics import PredictionEvaluator
from src.config import Config

## Load the data

In [2]:
config = Config()
variant_co2 = 'co2'
train_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/train.csv')).set_index(["year", config.additional_index])
test_df = pd.read_csv(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/test.csv')).set_index(["year", config.additional_index])

with open(os.path.join(config.output_cleaned_lstm, f'{variant_co2}/data_preprocessor_lstm.pkl'), 'rb') as f:
    data_preprocessor_lstm = pickle.load(f)

In [3]:
train_df

,,country_t-2,population_t-2,gdp_t-2,temperature_change_from_co2_t-2,cement_co2_t-2,coal_co2_t-2,flaring_co2_t-2,gas_co2_t-2,land_use_change_co2_t-2,oil_co2_t-2,...,population_t-1,gdp_t-1,temperature_change_from_co2_t-1,cement_co2_t-1,coal_co2_t-1,flaring_co2_t-1,gas_co2_t-1,land_use_change_co2_t-1,oil_co2_t-1,co2
year,country_index,,,,,,,,,,,,,,,,,,,,,
1930,Argentina,0,0.006943,0.008760,0.026846,0.000282,0.001872,0.000000,0.000253,0.189716,0.001601,...,0.007087,0.008291,0.02649,0.000370,0.001696,0.000000,0.000390,0.190037,0.001654,0.050794
1931,Argentina,0,0.007176,0.009169,0.026846,0.000423,0.001881,0.000000,0.000390,0.190037,0.001654,...,0.007303,0.007940,0.02649,0.000402,0.001655,0.000000,0.000386,0.170616,0.001586,0.046617
1932,Argentina,0,0.007394,0.008782,0.026846,0.000459,0.001835,0.000000,0.000386,0.170616,0.001586,...,0.007503,0.007374,0.02649,0.000564,0.001416,0.000000,0.000480,0.157812,0.002007,0.044323
1933,Argentina,0,0.007595,0.008158,0.026846,0.000644,0.001570,0.000000,0.000480,0.157812,0.002007,...,0.007686,0.007125,0.02649,0.000525,0.001291,0.000000,0.000655,0.150694,0.002278,0.042756
1934,Argentina,0,0.007779,0.007883,0.026846,0.000600,0.001431,0.000000,0.000655,0.150694,0.002278,...,0.007850,0.007466,0.02649,0.000540,0.001215,0.000000,0.000880,0.145852,0.002371,0.041755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003,Vietnam,44,0.059659,0.027092,0.026846,0.015970,0.005640,0.017690,0.002183,0.159475,0.010126,...,0.059873,0.026209,0.02649,0.017601,0.005604,0.013673,0.003490,0.148809,0.011342,0.055115
2004,Vietnam,44,0.060292,0.028933,0.026846,0.020100,0.006213,0.013673,0.003490,0.148809,0.011342,...,0.060508,0.028055,0.02649,0.019924,0.006518,0.016147,0.004431,0.157641,0.011685,0.057655
2005,Vietnam,44,0.060932,0.030970,0.026846,0.022752,0.007227,0.016147,0.004431,0.157641,0.011685,...,0.061146,0.030158,0.02649,0.021913,0.007093,0.015907,0.007456,0.160666,0.013424,0.052390


## Reshape the data

In [4]:
data_resherper = DataReshaperLSTM()
x_train, x_test, y_train, y_test = data_resherper.reshape_data(train_df, test_df)

## Build the model

In [5]:
input_shape = (x_train.shape[1], x_train.shape[2])
output_units = 1

model = Sequential([
    LSTM(50, input_shape=input_shape, return_sequences = True),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(output_units, activation='relu')
])

model.compile(optimizer="adam", loss="mean_squared_error")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             14200     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 34,451
Trainable params: 34,451
Non-trainable params: 0
_________________________________________________________________


## Train the model

In [6]:
history = model.fit(x_train, y_train, epochs=config.epochs, batch_size=config.batch_size)

Epoch 1/15
3103/3103 [==============================] - 3s 690us/step - loss: 0.0041
Epoch 2/15
3103/3103 [==============================] - 3s 869us/step - loss: 0.0016
Epoch 3/15
3103/3103 [==============================] - 2s 677us/step - loss: 0.0015
Epoch 4/15
3103/3103 [==============================] - 2s 730us/step - loss: 0.0011
Epoch 5/15
3103/3103 [==============================] - 2s 684us/step - loss: 0.0011
Epoch 6/15
3103/3103 [==============================] - 2s 675us/step - loss: 9.8044e-04
Epoch 7/15
3103/3103 [==============================] - 2s 679us/step - loss: 8.8436e-04
Epoch 8/15
3103/3103 [==============================] - 2s 677us/step - loss: 8.0008e-04
Epoch 9/15
3103/3103 [==============================] - 2s 676us/step - loss: 9.4063e-04
Epoch 10/15
3103/3103 [==============================] - 2s 677us/step - loss: 7.8468e-04
Epoch 11/15
3103/3103 [==============================] - 2s 703us/step - loss: 7.4377e-04
Epoch 12/15
3103/3103 [================

In [7]:
# Save the model
model.save(os.path.join(config.models_folder, f'{variant_co2}_lstm_model.h5'))

# Evaluate the model on the test set
loss = model.evaluate(x_test, y_test, verbose = 0)

## Predictions

In [8]:
train_predictions = model.predict(x_train)
test_predictions = model.predict(x_test)

inverted_data_predicted_train_y = data_preprocessor_lstm.inverse_transform_data(train_predictions, train_predictions.shape[0], train_df.shape[1]-2)
inverted_data_train_y = data_preprocessor_lstm.inverse_transform_data(y_train, train_predictions.shape[0], train_df.shape[1]-2)

inverted_data_predicted_test_y = data_preprocessor_lstm.inverse_transform_data(test_predictions, test_predictions.shape[0], test_df.shape[1]-2)
inverted_data_test_y = data_preprocessor_lstm.inverse_transform_data(y_test, test_predictions.shape[0], test_df.shape[1]-2)

26/26 [==============================] - 0s 415us/step


In [9]:
train_df_reset = train_df.reset_index()
test_df_reset = test_df.reset_index()

co2_predicted_train = inverted_data_predicted_train_y[:, -1]
co2_actual_train = inverted_data_train_y[:, -1]
co2_predicted_test = inverted_data_predicted_test_y[:, -1]
co2_actual_test = inverted_data_test_y[:, -1]

# Create DataFrames for train and test
train_results = pd.DataFrame({
    "country": train_df_reset["country_index"].values,
    "year": train_df_reset["year"].values,
    "co2_predicted": co2_predicted_train,
    "co2_actual": co2_actual_train
})

test_results = pd.DataFrame({
    "country": test_df_reset["country_index"].values,
    "year": test_df_reset["year"].values,
    "co2_predicted": co2_predicted_test,
    "co2_actual": co2_actual_test
})


train_results.to_csv(os.path.join(config.predictions_lstm, f'{variant_co2}_train.csv'))
test_results.to_csv(os.path.join(config.predictions_lstm, f'{variant_co2}_test.csv'))

## Charts

In [10]:
charts = ModelCharts(train_results, test_results)

#Line plot
charts.generate_line_plot(variant = variant_co2, model_output_file=config.predictions_lstm)
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='train')
charts.generate_line_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='test')

#Scatter plot
charts.generate_scatter_plot(variant = variant_co2, model_output_file=config.predictions_lstm)
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='train')
charts.generate_scatter_plot_one_dataset(variant = variant_co2, model_output_file=config.predictions_lstm, dataset_type='test')

## Metrics

In [11]:
evaluator = PredictionEvaluator()
evaluator.evaluate(train_results, test_results, actual_col='co2_actual', predicted_col='co2_predicted', variant = variant_co2, model_output_file=config.metrics_lstm)